# S&P 500 Analysis

## Import the Packages, Functions and Data

In [1]:
import pandas as pd
from EDA_functions import *
from Baseline_functions import *
from DQN_functions import *
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_pickle('data/SP500.pkl')

## Outlier Removal 


In [3]:
df['daily_return'] = df['Close'].pct_change()
df['daily_return'] = df['daily_return'].fillna(0)
#mean plus 3sigma value for absolute daily returns
mean = df['daily_return'].mean()
std = df['daily_return'].std()
upper_limit = mean + 2*std
lower_limit = mean - 2*std
print(f'Mean: {mean}, Std: {std}, Upper Limit: {upper_limit}, Lower Limit: {lower_limit}')
#number of rows that are outside the 3 sigma range
print(f'Number of rows outside 2 sigma range: {len(df[(df["daily_return"]>upper_limit) | (df["daily_return"]<lower_limit)])}')
print(f'Percentage of rows outside 2 sigma range: {len(df[(df["daily_return"]>upper_limit) | (df["daily_return"]<lower_limit)])/len(df)*100}% ')

#if daily return is outside 3 sigma range, replace all the values with previous day's values
for i in range(1,len(df)):
    if df['daily_return'][i]>upper_limit or df['daily_return'][i]<lower_limit:
        df['Close'][i] = df['Close'][i-1]
        df['Open'][i] = df['Open'][i-1]
        df['High'][i] = df['High'][i-1]
        df['Low'][i] = df['Low'][i-1]
        df['Volume'][i] = df['Volume'][i-1]
        df['daily_return'][i] = 'NaN'

Mean: 0.00037873998112499885, Std: 0.012040180200659603, Upper Limit: 0.024459100382444206, Lower Limit: -0.023701620420194205
Number of rows outside 2 sigma range: 244
Percentage of rows outside 2 sigma range: 4.848003179018478% 


## DQN

### Training

In [4]:
df_base = df[['Open','High','Low','Close','Volume']]
#include 26 and 12 ema
# df_base['26ema'] = df_base['Close'].ewm(span=26).mean()
# df_base['12ema'] = df_base['Close'].ewm(span=12).mean()
#fill na as 0
# df_base = df_base.dropna()
# df_base = df_base[['Open','High','Low','Close','Volume','26ema','12ema']]

#first 80% of the data is train
df_train = df_base.iloc[:int(len(df_base)*0.8)]
#last 20% of the data is test
df_test = df_base.iloc[int(len(df_base)*0.8):]

In [5]:
display(df_train.head())

display(df_test.head())

,Open,High,Low,Close,Volume
Date,,,,,
2004-03-31,1127.000000,1130.829956,1121.459961,1126.209961,1560700000
2004-04-01,1126.209961,1135.670044,1126.199951,1132.170044,1560700000
2004-04-02,1132.170044,1144.810059,1132.170044,1141.810059,1629200000
2004-04-05,1141.810059,1150.569946,1141.640015,1150.569946,1413700000
2004-04-06,1150.569946,1150.569946,1143.300049,1148.160034,1397700000


,Open,High,Low,Close,Volume
Date,,,,,
2020-03-30,2457.770020,2571.419922,2407.530029,2475.560059,8300010000
2020-03-31,2614.689941,2641.389893,2571.149902,2584.590088,6576210000
2020-04-01,2614.689941,2641.389893,2571.149902,2584.590088,6576210000
2020-04-02,2458.540039,2533.219971,2455.790039,2526.899902,6464190000
2020-04-03,2514.919922,2538.179932,2459.959961,2488.649902,6096970000


In [6]:
print(df_train.shape, df_test.shape)
print(df_train.shape[0] + df_test.shape[0])
print(df.shape[0])

(4026, 5) (1007, 5)
5033
5033


In [7]:
window_size = 26
states = create_states(df_train, window_size)
test_states = create_states(df_test, window_size)
input_dim_conv = states.shape[2]
output_dim = 3
print("Shape of states:", states.shape)

Shape of states: (4000, 26, 5)


In [8]:
model = ConvDQN(input_dim_conv, output_dim, window_size)
memory = ReplayMemory(50000)
agent = DQNAgent(input_dim_conv, output_dim, window_size,model, lr=0.0001, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.9995)

In [9]:
%%time
log_train = train_agent(agent, states, 800, batch_size=32)

Episode 1/800, Total Reward: -410.41258374023437, Loss: 27.569311510209893
Episode 2/800, Total Reward: -105.87585316354982, Loss: 25.077892425076048
Episode 3/800, Total Reward: -174.76006527158205, Loss: 25.053354631031144
Episode 4/800, Total Reward: 2.5557238469055332, Loss: 22.30231566513968
Episode 5/800, Total Reward: -87.73936880469728, Loss: 21.532307095093156
Episode 6/800, Total Reward: -562.980224609375, Loss: 19.228449773322854
Episode 7/800, Total Reward: 0.0, Loss: 16.95606032087919
Episode 8/800, Total Reward: 0.0, Loss: 13.142727182994825
Episode 9/800, Total Reward: -456.01398193359375, Loss: 11.751624216903778
Episode 10/800, Total Reward: 0.0, Loss: 9.474289242671619
Episode 11/800, Total Reward: -1.1228307005467105e-20, Loss: 5.53363112738736
Episode 12/800, Total Reward: -3.450456311203726e-05, Loss: 6.142492440547109
Episode 13/800, Total Reward: -0.0013180280729625149, Loss: 5.6497945450827505
Episode 14/800, Total Reward: -3.065604320471928, Loss: 5.49331647748

In [ ]:
log_train.to_csv('SP500/log_train_CNN_updated_reward_800ep.csv', index=False)

In [ ]:
#save the model
torch.save(agent.model.state_dict(), 'SP500/DQN_CNN_updated_reward_800ep.pth')


In [ ]:
print("Training without Hold penalty")
log_train_no_hold = train_agent_hold(agent, states, 800, batch_size=32)

In [ ]:
log_train_no_hold.to_csv('SP500/log_train_CNN_updated_reward_MA_no_hold_800ep.csv', index=False)

#save the model
torch.save(agent.model.state_dict(), 'SP500/DQN_CNN_updated_reward_MA_no_hold_800ep.pth')

### Evaluating and Testing

In [ ]:
log_train = pd.read_csv('SP500/log_train_CNN_updated_reward_MA_800ep.csv')
log_train_no_hold = pd.read_csv('SP500/log_train_CNN_updated_reward_MA_no_hold_800ep.csv')

In [ ]:
log_train.rename(columns={'Reward':'Training_reward'}, inplace=True)
log_train_no_hold.rename(columns={'Reward':'Training_reward_no_hold'}, inplace=True)

In [ ]:
%%capture

reward =[]
for i in range(1,log_train['Episode'].max()+1):
    last_episode = log_train[log_train['Episode'] == i]
    last_episode.reset_index(drop=True, inplace=True)
    last_episode['Close'] = last_episode['Price'].to_list()
    reward.append(list(capital_calculation(last_episode,'Action'))[-1]-100);

reward_no_hold =[]
for i in range(1,log_train_no_hold['Episode'].max()+1):
    last_episode = log_train_no_hold[log_train_no_hold['Episode'] == i]
    last_episode.reset_index(drop=True, inplace=True)
    last_episode['Close'] = last_episode['Price'].to_list()
    reward_no_hold.append(list(capital_calculation(last_episode,'Action'))[-1]-100);

In [ ]:
log_train['Reward'] = log_train['Episode'].apply(lambda x: reward[x-1])
log_train_no_hold['Reward'] = log_train_no_hold['Episode'].apply(lambda x: reward_no_hold[x-1])

In [ ]:
reward_filter(reward)

In [ ]:
reward_filter(reward_no_hold)

In [ ]:
plot_training(log_train)

In [ ]:
plot_training(log_train_no_hold)

In [ ]:
action_episode_df = create_action_episode_df(log_train)
action_episode_df_no_hold = create_action_episode_df(log_train_no_hold)

In [ ]:
action_episode_df = action_episode_df.apply(pd.Series.value_counts).transpose().reset_index()
action_episode_df_no_hold = action_episode_df_no_hold.apply(pd.Series.value_counts).transpose().reset_index()

In [ ]:
#get value counts of all columns in the DataFrame
action_episode_df[['Buy']].plot()

In [ ]:
action_episode_df_no_hold[['Buy']].plot()

In [ ]:
# Initialize the agent
test_state_size = test_states.shape[2]
action_size = 3
model = ConvDQN(input_dim_conv, output_dim, window_size)
agent = DQNAgent(test_state_size, output_dim, window_size,model, lr=0.0001, gamma=0.95, epsilon=0, epsilon_min=0, epsilon_decay=0.9995)

In [ ]:
test_state_size

In [ ]:
model_path = 'SP500/DQN_CNN_updated_reward_MA_800ep.pth'
agent.model.load_state_dict(torch.load(model_path))

# Ensure the model is in evaluation mode
agent.model.eval()

In [ ]:
evaluation_log = evaluate_agent(agent, test_states)

In [ ]:
evaluation_log['Close'] = evaluation_log['Price']
evaluation_log['Capital'] = capital_calculation(evaluation_log, 'Action')

In [ ]:
plot_dual_axis(evaluation_log)

In [ ]:
print(f"Return without DQN: {(evaluation_log['Close'].iloc[-1] - evaluation_log['Close'].iloc[0])/evaluation_log['Close'].iloc[0] }%")
print(f"Return with DQN: {(evaluation_log['Capital'].iloc[-1] - 100)/100 }%")

In [ ]:
all_states = create_states(df_base, window_size)
all_states_eval = evaluate_agent(agent, all_states)

In [ ]:
all_states_eval['Close'] = all_states_eval['Price']
all_states_eval['Capital'] = capital_calculation(all_states_eval, 'Action')
all_states_eval

In [ ]:
print(f'Return without trading: {(all_states_eval["Close"].iloc[-1] - all_states_eval["Close"].iloc[0])/all_states_eval["Close"].iloc[0]}')
print(f"Return on Investment: {(all_states_eval['Capital'].iloc[-1] - 100)/100}")

In [ ]:
plot_dual_axis(all_states_eval)

In [ ]:
evaluation_log['Date']= df_test.tail(evaluation_log.shape[0]).index

In [ ]:
evaluation_log

##